In [7]:
# SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.engine import reflection

#import egnine
from config import engine_url

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Pandas
import pandas as pd

In [8]:
engine = create_engine(engine_url)
con = engine.connect()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'cw-spmb.ct4csmrmjrt7.us-west-2.rds.amazonaws.com' (timed out)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
firm_id = input('Enter the Firm ID')
# sql to get addresses
sql_address = """ SELECT p.id as person_id, 
        pdata.street, 
        pdata.street_2, pdata.city,
        pdata.state,
        pdata.postal_code,
        pdata.country,
        reg.name as region,
        if(pdata.id = p.preferred_address_id, "preferred","") as preferred
        from `people` as p
        inner join person_addresses as pdata on p.id = pdata.person_id
        left join regions as reg on pdata.`region_id` = reg.id
        left join location_types as loc on pdata.location_type_id = loc.id
        where p.firm_id = %s
        order by person_id """
address_df = pd.read_sql_query(sql_address, con, params=[firm_id])
address_df.to_csv("address.csv", sep='\t', encoding='utf-8')

In [ ]:
firm_id = input('Enter Firm ID')
# SQL to split out email
sql_email = """ SELECT p.id as person_id,
            pdata.`address` as email,
            loc.name as type,
            if(pdata.id = p.preferred_email_address_id, "preferred","") as preferred
            from `people` as p
            inner join person_email_addresses as pdata on p.id = pdata.person_id
            left join location_types as loc on pdata.location_type_id = loc.id
            where p.firm_id = %s
            order by person_id"""
email_df = pd.read_sql_query(sql_email, con, params=[firm_id])
email_df.to_csv("email.csv", sep='\t', encoding='utf-8')

In [5]:
firm_id = input('Enter Firm ID')
# SQL to split out Phone Number
sql_phone = """SELECT p.id as person_id,
            pdata.digits as phone,
            pdata.extension as extension,
            loc.name as type,
            if(pdata.id = p.preferred_phone_number_id, "preferred","") as preferred
            from `people` as p
            inner join person_phone_numbers as pdata on p.id = pdata.person_id
            left join location_types as loc on pdata.location_type_id = loc.id
            where p.firm_id = %s
            order by person_id"""
phone_df = pd.read_sql_query(sql_phone, con, params=[firm_id])
phone_df.to_csv("phone.csv", sep='\t', encoding='utf-8')

Enter Firm ID1


In [ ]:
# address_df

In [ ]:
# insp = reflection.Inspector.from_engine(engine)
# print(insp.get_table_names())

In [ ]:
# insp.get_columns('projects', schema=None)

In [ ]:
firm_id = input('Enter Firm ID')

In [15]:
firm_id = 137
month = 4
year = 2018
cw_em = '%@clockworkrecruiting.com'
sql_events = """select p.name, 
            u.email, 
            vu.primary_role,
            e.actor_id, 
            e.type, 
            if(nt.name is not null, nt.name, "") as note_type,
            date(e.updated_at) as event_date,
            count(e.id)
            from events e
            left join users u on e.actor_id = u.id
            left join vw_user_roles vu on u.id = vu.user_id
            left join people p on p.id = u.person_id
            left join note_categories nt on e.`subcontext_id` = nt.id
            where month(e.created_at) = %s
            and year(e.created_at) = %s
            and e.firm_id = %s
            and vu.email not like %s
            group by e.actor_id, e.type, event_date"""
events_df = pd.read_sql_query(sql_events, con, params=(month, year, firm_id, cw_em))
events_df.to_csv('events.csv', sep='\t', encoding='utf-8')